In [1]:
import sklearn
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("features.csv")

In [3]:
dfx = np.array(dataset)
dfy = np.array(pd.read_csv("dataset.csv")["Clickbait"])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=1)

In [5]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation="relu", solver="sgd", hidden_layer_sizes=(7), random_state=1)

In [6]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=7, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [15]:
mlp_y_pred = mlp.predict(X_test)

In [16]:
from sklearn import metrics
metrics.accuracy_score(y_test, mlp_y_pred)

0.9025

In [17]:
from sklearn import svm

In [22]:
clf = svm.SVC(kernel='rbf')

In [23]:
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
svm_y_pred = clf.predict(X_test)

In [25]:
from sklearn import metrics
metrics.accuracy_score(y_test, svm_y_pred)

0.9041666666666667